In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import random as rd

In [ ]:
class Estado():
  def __init__(self):
    self.recompensa = 0
    self.valor = 0
    self.politica = [0.25, 0.25, 0.25, 0.25]

In [ ]:
class Grid():

  def __init__(self, tamanho, montanhas=3, areias=3):
    self.montanhas = montanhas
    self.areias = areias
    self.grid = self.montar_grid(tamanho)

  def montar_grid(self, tamanho):
    grid = []
    for i in range(tamanho):
      grid.append([])
      for j in range(tamanho):
        grid[i].append(Estado())

    self.definir_recompensas(grid, self.montanhas, self.areias)

    return grid

  def definir_recompensas(self, grid, montanhas=3, areias=3):
    n_montanha = 0
    while n_montanha < montanhas:
      lat = rd.randint(0, len(grid)-1)
      lon = rd.randint(0, len(grid)-1)
      if not (lat == 0 and lon == 0) and not (lat == len(grid)-1 and lon == len(grid)-1):
        if grid[lat][lon].recompensa == 0:
          grid[lat][lon].recompensa = -5
          n_montanha += 1

    n_areia = 0
    while n_areia < areias:
      lat = rd.randint(0, len(grid)-1)
      lon = rd.randint(0, len(grid)-1)
      if not (lat == 0 and lon == 0) and not (lat == len(grid)-1 and lon == len(grid)-1):
        if grid[lat][lon].recompensa == 0:
          grid[lat][lon].recompensa = -10
          n_areia += 1

    grid[7][7].recompensa = 10

In [ ]:
class Agente():

  def __init__(self, grid):
    self.fator_de_desconto = 0.9
    self.grid = grid
    self.lat_atual = 0
    self.lon_atual = 0
    self.politica_da_acao = self.grid[self.lat_atual][self.lon_atual].politica[0]
    self.recompensa_atual = self.grid[self.lat_atual][self.lon_atual].recompensa
    self.valor_atual = self.grid[self.lat_atual][self.lon_atual].valor
    self.definicao_dos_valores()

  def definicao_dos_valores(self):
    tolerancia = 0.0001
    valor_anterior = 1
    valor_atual = 0
    while abs(valor_anterior - valor_atual) >= tolerancia:
      for i in range(len(self.grid)):
        self.lat_atual = i
        for j in range(len(self.grid)):
          self.lon_atual = j
          valor_anterior = self.grid[self.lat_atual][self.lon_atual].valor
          self.grid[self.lat_atual][self.lon_atual].valor = self.valor_do_estado(self.lat_atual, self.lon_atual)
          valor_atual = self.grid[self.lat_atual][self.lon_atual].valor
        
    self.lat_atual = 0
    self.lon_atual = 0

  def valor_do_estado(self, lat, lon):
    valor = 0
    self.lat_atual = lat
    self.lon_atual = lon
    for i in range(4):
      self.realizar_acoes(i)
      valor += self.politica_da_acao * (self.recompensa_atual + self.fator_de_desconto * self.valor_atual)
    
    return valor

  def realizar_acoes(self, acao):
    self.politica_da_acao = self.grid[self.lat_atual][self.lon_atual].politica[acao]
    if acao == 0:
      return self.cima(self.lat_atual, self.lon_atual)
    elif acao == 1:
      return self.baixo(self.lat_atual, self.lon_atual)
    elif acao == 2:
      return self.esquerda(self.lat_atual, self.lon_atual)
    elif acao == 3:
      return self.direita(self.lat_atual, self.lon_atual)

  def cima(self, lat, lon):
    if lat == 0:
      self.valor_atual = self.grid[lat][lon].valor
      self.recompensa_atual = -1
      return (lat, lon)
    else:
      self.valor_atual = self.grid[lat-1][lon].valor
      self.recompensa_atual = self.grid[lat-1][lon].recompensa
      return (lat-1, lon)
  
  def baixo(self, lat, lon):
    if lat == len(self.grid)-1:
      self.valor_atual = self.grid[lat][lon].valor
      self.recompensa_atual = -1
      return (lat, lon)
    else:
      self.valor_atual = self.grid[lat+1][lon].valor
      self.recompensa_atual = self.grid[lat+1][lon].recompensa
      return (lat+1, lon)
  
  def esquerda(self, lat, lon):
    if lon == 0:
      self.valor_atual = self.grid[lat][lon].valor
      self.recompensa_atual = -1
      return (lat, lon)
    else:
      self.valor_atual = self.grid[lat][lon-1].valor
      self.recompensa_atual = self.grid[lat][lon-1].recompensa
      return (lat, lon-1)

  def direita(self, lat, lon):
    if lon == len(self.grid)-1:
      self.valor_atual = self.grid[lat][lon].valor
      self.recompensa_atual = -1
      return (lat, lon)
    else:
      self.valor_atual = self.grid[lat][lon+1].valor
      self.recompensa_atual = self.grid[lat][lon+1].recompensa
      return (lat, lon+1)
    
  def step(self):
    rota = []

    while self.lat_atual != 7 and self.lon_atual != 7:
      posicao_atual = self.grid[self.lat_atual][self.lon_atual]
      rota.append(posicao_atual)

      valores_acao = []
      for acao in range(4):
        prox_lat, prox_lon = self.realizar_acoes(acao)
        valores_acao.append(self.recompensa_atual + self.grid[prox_lat][prox_lon].valor)
      melhor_acao = valores_acao.index(max(valores_acao))
      valores_acao.clear()

      self.lat_atual, self.lon_atual = self.realizar_acoes(melhor_acao)
    
    return rota
